https://colab.research.google.com/drive/1UOjgEpngaiXYA3BKwc3zKUbofDbZTdYO

In [0]:
import numpy as np
import pandas as pd

In [2]:
!wget https://www.dropbox.com/s/thbppaeock1yx7z/u.data

--2019-10-01 15:10:50--  https://www.dropbox.com/s/thbppaeock1yx7z/u.data
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/thbppaeock1yx7z/u.data [following]
--2019-10-01 15:10:50--  https://www.dropbox.com/s/raw/thbppaeock1yx7z/u.data
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf9132efa3742dc6535a81cff4b.dl.dropboxusercontent.com/cd/0/inline/AplXd2-yv_aj0o9F08RvI2BnLXXxa3rVOKfNdi_9zq2TkfidYxVeuaIlp8MMskYzHT1IhhvNdPeyFu44KcYgIPTIwpB0gLYfWRzJKI8Xcgjkug/file# [following]
--2019-10-01 15:10:51--  https://ucf9132efa3742dc6535a81cff4b.dl.dropboxusercontent.com/cd/0/inline/AplXd2-yv_aj0o9F08RvI2BnLXXxa3rVOKfNdi_9zq2TkfidYxVeuaIlp8MMskYzHT1IhhvNdPeyFu44KcYgIPTIwpB0gLYfWRzJKI8Xcgjkug/file
Resolving ucf9132efa3742dc6535a81c

In [0]:
column_names = ['User', 'Movies', 'rating', 'timestamp']
ratings = pd.read_csv('/content/u.data', sep='\t', names=column_names)

In [4]:
ratings.head()

,User,Movies,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
ratings['User2']=ratings['User'].astype('category')
ratings['Movies2']=ratings['Movies'].astype('category')

In [6]:
ratings.head()

,User,Movies,rating,timestamp,User2,Movies2
0,196,242,3,881250949,196,242
1,186,302,3,891717742,186,302
2,22,377,1,878887116,22,377
3,244,51,2,880606923,244,51
4,166,346,1,886397596,166,346


In [7]:
ratings.shape

(100000, 6)

In [0]:
users = ratings.User.unique()
articles = ratings.Movies.unique()

In [9]:
len(users)

943

In [10]:
len(articles)

1682

In [11]:
len(ratings['Movies'][:100000].unique())

1682

In [0]:
ratings['User2']=ratings['User'].astype('category')
ratings['Movies2']=ratings['Movies'].astype('category')

In [0]:
userid2idx = {o:i for i,o in enumerate(users)}
articlesid2idx = {o:i for i,o in enumerate(articles)}

In [0]:
idx2userid = {i:o for i,o in enumerate(users)}
idx2articlesid = {i:o for i,o in enumerate(articles)}

In [15]:
print(articlesid2idx[313],articlesid2idx[60],articlesid2idx[61])

719 330 169


In [0]:
ratings['Movies2'] = ratings.Movies.apply(lambda x: articlesid2idx[x])
ratings['User2'] = ratings.User.apply(lambda x: userid2idx[x])

In [17]:
ratings.head()

,User,Movies,rating,timestamp,User2,Movies2
0,196,242,3,881250949,0,0
1,186,302,3,891717742,1,1
2,22,377,1,878887116,2,2
3,244,51,2,880606923,3,3
4,166,346,1,886397596,4,4


In [18]:
ratings['Movies'].unique()

array([ 242,  302,  377, ..., 1637, 1630, 1641])

In [0]:
n_users = ratings.User.nunique()
n_articles = ratings.Movies.nunique()

In [20]:
n_users

943

In [21]:
import keras.backend as K
def rmse(y_true,y_pred):
  score = K.sqrt(K.mean(K.pow(y_true - y_pred, 2)))
  return score

Using TensorFlow backend.


In [0]:
from keras.layers import Input, Embedding, Dense, Dropout, merge, Flatten
from keras.models import Model

In [0]:
def embedding_input(name,n_in,n_out):
  inp = Input(shape=(1,),dtype='int64',name=name)
  return inp, Embedding(n_in,n_out,input_length=1)(inp)

In [24]:
n_factors = 100
u = embedding_input('user_in', n_users, n_factors)
a = embedding_input('article_in', n_articles, n_factors)

In [0]:
n_factors = 100
user_in, u = embedding_input('user_in', n_users, n_factors)
article_in, a = embedding_input('article_in', n_articles, n_factors)

In [26]:
user_in

<tf.Tensor 'user_in_1:0' shape=(?, 1) dtype=int64>

In [27]:
user_in

<tf.Tensor 'user_in_1:0' shape=(?, 1) dtype=int64>

In [0]:
from keras.layers import dot

In [0]:
??dot

In [32]:
u.shape, a.shape

(TensorShape([Dimension(None), Dimension(1), Dimension(100)]),
 TensorShape([Dimension(None), Dimension(1), Dimension(100)]))

In [46]:
dot([u,a], axes=2)

<tf.Tensor 'dot_4/MatMul:0' shape=(?, 1, 1) dtype=float32>

In [0]:
x = dot([u,a], axes=2)

In [0]:
x=Flatten()(x)
x = Dense(500, activation='relu')(x)
x = Dense(1)(x)

In [0]:
from keras import optimizers

In [50]:
model = Model([user_in,article_in],x)
adam = optimizers.Adam(lr=0.01)
model.compile(adam,loss='mse',metrics=[rmse])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
article_in (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 100)       94300       user_in[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 100)       168200      article_in[0][0]                 
____________________________________________________________________________________________

In [0]:
model.fit([ratings.User2,ratings.Movies2], ratings.rating/5, epochs=50,batch_size=128)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
articlesid2idx[59]

574

In [56]:
np.argmax(cosine_similarity(model.get_weights()[1][574].reshape(1,-1),model.get_weights()[1][:574].reshape(574,100)))

169

In [57]:
idx2articlesid[169]

61